# Курсовая работа № 2
## Аналитика. Начальный уровень

Курсовая работа состоит из двух частей – обязательной и дополнительной. **Для зачета необходимо выполнение только первой части.** Выполнение второй части может потребовать дополнительные знания Python.

- [Часть первая](#Часть-первая)
- [Часть вторая](#Часть-вторая)

## Часть первая


Перед вами стоит задача – подготовить аналитический отчет для HR-отдела. На основании проведенной аналитики предполагается составить рекомендации для отдела кадров по стратегии набора персонала, а также по взаимодействию с уже имеющимися сотрудниками.
<br><br> В базе данных лежит набор таблиц, которые содержат данные о сотрудниках вымышленной компании.
Сделайте обзор штата сотрудников компании. Составьте набор предметов исследования, а затем проверьте их на данных. Вся аналитика должна быть выполена с помощью SQL. Впоследствии данные можно визуализировать, однако финальные датафреймы для графиков также должны быть подготовлены с помощью SQL. <br><br>

Примеры гипотез:
1. Есть зависимость между `perfomance score` и тем, под чьим руководством работает сотрудник.
2. Есть зависимость между продолжительностью работы в компании и семейным положением сотрудника.
2. Есть зависимость между продолжительностью работы в компании и возрастом сотрудника.

<br><br>
Параметры для подключения следующие: хост – `dsstudents.skillbox.ru`, порт – `5432`, имя базы данных – `human_resources`, пользователь – `readonly`, пароль – `6hajV34RTQfmxhS`. Таблицы, доступные для анализа, – `hr_dataset`, `production_staff`, `recruiting_costs`, `salary_grid`.

In [108]:
import psycopg2
import pandas as pd
import numpy as np

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "human_resources",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}

In [109]:
def move(command):
    conn = psycopg2.connect(**pg_connection)
    cursor = conn.cursor()
    sql_str = command
    cursor.execute(sql_str)
    col = [a[0] for a in cursor.description]
    data = np.array(cursor.fetchall())
    df = pd.DataFrame(data, columns=col)
    cursor.close()
    conn.commit()
    return df

In [124]:
# Есть зависимость между среднем 'Days Employed' и полом
# Есть зависимость между средней зарплатой и полом
move("""
    SELECT sex, COUNT(*), AVG("Pay Rate") as rate, avg(age) as age,
    AVG("Days Employed") as "Days Employed" FROM hr_dataset
    GROUP BY sex
;""")

,sex,count,rate,age,Days Employed
0,Female,177,29.4721,38.9604519774011299,1313.4576271186440678
1,Male,133,33.6971,38.7443609022556391,1272.9548872180451128


In [127]:
# Есть зависимость между количеством людей работающих в департаменте и 'Days Employed'
move("""
    SELECT department, COUNT(*) as count, AVG("Pay Rate") as rate, avg(age) as age,
    AVG("Days Employed") as "Days Employed"
    FROM hr_dataset
    GROUP BY department
    ORDER BY count
;""")

,department,count,rate,age,Days Employed
0,Executive Office,1,80,63.0000000000000000,1973.0000000000000000
1,Admin Offices,10,31.896,32.3000000000000000,1420.7000000000000000
2,Software Engineering,10,48.665,35.9000000000000000,1417.8000000000000000
3,Sales,31,55.5242,39.0967741935483871,1621.5483870967741935
4,IT/IS,50,45.7896,38.0200000000000000,981.8000000000000000
5,Production,208,23.0863,39.3798076923076923,1308.0240384615384615


In [112]:
# Есть зависимость между средней зарплатой в департаменте и 'Days Employed'
move("""
    SELECT department, COUNT(marriedid), AVG("Pay Rate") as rate, avg(age) as age,
    AVG("Days Employed") as "Days Employed" FROM hr_dataset
    GROUP BY department
    ORDER BY rate
;""")

,department,count,rate,age,Days Employed
0,Production,208,23.0863,39.3798076923076923,1308.0240384615384615
1,Admin Offices,10,31.896,32.3000000000000000,1420.7000000000000000
2,IT/IS,50,45.7896,38.0200000000000000,981.8000000000000000
3,Software Engineering,10,48.665,35.9000000000000000,1417.8000000000000000
4,Sales,31,55.5242,39.0967741935483871,1621.5483870967741935
5,Executive Office,1,80,63.0000000000000000,1973.0000000000000000


In [123]:
# Есть зависимость между "Position" и медианной зарплатой
move("""
    SELECT  "Position", "Salary Mid" FROM salary_grid
    ORDER BY "Salary Mid"
;""")

,Position,Salary Mid
0,Production Technician I,40000
1,Administrative Assistant,40000
2,Sr. Administrative Assistant,45000
3,Production Technician II,48000
4,Accountant I,51425
5,Lead Production Technician,55000
6,Accountant II,62158
7,Network Engineer,66850
8,Database Administrator,68306
9,Sr. Accountant,76988


In [114]:
#Есть зависимость между 'Employment Source' и средствами потраченными на рекрутинг
move("""
    SELECT * FROM recruiting_costs
    ORDER BY "Total"
;""")

,id,Employment Source,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,13,On-line Web application,0,0,0,0,0,0,0,0,0,0,0,0,0
1,22,Word of Mouth,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Company Intranet - Partner,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20,Vendor Referral,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,Employee Referral,0,0,0,0,0,0,0,0,0,0,0,0,0
5,6,Glassdoor,0,0,0,0,0,0,0,0,0,0,0,0,0
6,7,Information Session,0,0,0,0,0,0,0,0,0,0,0,0,0
7,8,Internet Search,0,0,0,0,0,0,0,0,0,0,0,0,0
8,17,Professional Society,100,100,100,100,100,100,100,100,100,100,100,100,1200
9,15,Pay Per Click,110,110,60,121,110,109,130,146,105,109,105,110,1323


In [117]:
# Есть небольшая зависимость между рассой и зарплатой
move("""
    SELECT "Race Desc", AVG(Cast("Pay" as decimal)) as pay FROM production_staff
    where "Race Desc" is not null
    GROUP BY "Race Desc"
    ORDER BY pay
;""")

,Race Desc,pay
0,Two or more races,18.5260000000000000
1,American Indian or Alaska Native,21.8333333333333333
2,Asian,22.8045454545454545
3,White,22.9920714285714286
4,Black or African American,23.3419354838709677
5,Hispanic,32.0000000000000000


In [150]:
# Есть зависимость между именем менеджера и средней зарплатой коллектива
move("""
    SELECT "Manager Name", AVG(Cast("Pay" as decimal)) as pay, COUNT(*) FROM production_staff
    where "Manager Name" is not null
    GROUP BY "Manager Name"
    ORDER BY pay
;""")

,Manager Name,pay,count
0,David Stanley,19.9047619047619048,21
1,Kelley Spirea,20.7618181818181818,22
2,Webster Butler,20.7833333333333333,21
3,Ketsia Liebig,21.0000000000000000,21
4,Kissy Sullivan,21.1250000000000000,22
5,Elijiah Gray,21.1626086956521739,23
6,Amy Dunn,21.2857142857142857,21
7,Michael Albert,21.3043478260869565,23
8,Brannon Miller,21.7023809523809524,21
9,Elisa Bramante,49.6071428571428571,14


## Часть вторая

Перед вами стоит задача – подготовить аналитический ответ для SMM-отдела компании Skillbox. <br> Объектом анализа является  [паблик Skillbox Вконтакте](https://vk.com/skillbox_education). <br> <br> 
Подключитесь к  API VK и выгрузите посты со стены паблика Skillbox за интересующий период (определите самостоятельно и обоснуйте). Проанализируйте влияние различных факторов (например, времени публикации) на вовлеченность пользователей (количество лайков, комментариев, голосов в опросах). Сделайте аналитику по рубрикам (примеры рубрик: дизайн-битва, игра по управлению), которые есть в паблике. Выбрать нужные посты можно с помощью регулярных выражений. Составьте перечень рекомандаций для SMM-отдела по итогам анализа. <br> <br> 

Дополнительные инструкции по работе с API VK расположены [здесь](https://nbviewer.jupyter.org/github/vikaborel/dstutorials/blob/master/Дополнительные%20инструкции.ipynb).

In [0]:
### YOUR CODE HERE ###


